In [1]:
import torch
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

torch.manual_seed(777) # reproducibility

In [2]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [3]:
# MNIST dataset
mnist_train = dsets.MNIST(root = 'MNIST_data/',
                          train = True,
                          transform = transforms.ToTensor(),
                          download = True)

mnist_test = dsets.MNIST(root = 'MNIST_data/',
                         train = False,
                         transform = transforms.ToTensor(),
                         download = True)

In [4]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True)

In [5]:
# model
model = torch.nn.Linear(784, 10, bias = True)

In [6]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss() # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [7]:
# train my model
for epoch in range(training_epochs):
  avg_cost = 0
  total_batch = len(mnist_train) // batch_size

  for i, (batch_xs, batch_ys) in enumerate(data_loader):
    # reshape input image into [batch_size by 784]
    X = Variable(batch_xs.view(-1, 28 * 28))
    Y = Variable(batch_ys) # label is not one-hot encoded

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
  
  print("[Epoch: {:>4}] cost = {:>.9}".format(epoch + 1, avg_cost.data))

print('Learning Finished')

[Epoch:    1] cost = 0.614322782
[Epoch:    2] cost = 0.344514906
[Epoch:    3] cost = 0.308487236
[Epoch:    4] cost = 0.291329682
[Epoch:    5] cost = 0.281550318
[Epoch:    6] cost = 0.274611503
[Epoch:    7] cost = 0.269422829
[Epoch:    8] cost = 0.264997184
[Epoch:    9] cost = 0.261648178
[Epoch:   10] cost = 0.259036899
[Epoch:   11] cost = 0.256706268
[Epoch:   12] cost = 0.254275382
[Epoch:   13] cost = 0.252618015
[Epoch:   14] cost = 0.251062483
[Epoch:   15] cost = 0.249662459
Learning Finished


In [8]:
# Test model and check accuracy
X_test = Variable(mnist_test.test_data.view(-1, 28 * 28).float())
Y_test = Variable(mnist_test.test_labels)

prediction = model(X_test)
correct_prediction = (torch.max(prediction.data, 1)[1] == Y_test.data)
accuracy = correct_prediction.float().mean()
print('Accuracy:', accuracy)

Accuracy: tensor(0.9072)


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [9]:
# Get one and predict
r = random.randint(0, len(mnist_test) - 1)
X_single_data = Variable(mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float())
Y_single_data = Variable(mnist_test.test_labels[r:r + 1])

print("Label: ", Y_single_data.data)
single_prediction = model(X_single_data)
print("Prediction: ", torch.max(single_prediction.data, 1)[1])

Label:  tensor([7])
Prediction:  tensor([7])
